# Merge Salinity (Monthly) with Daily Weather Data

Gộp file độ mặn (time theo tháng) với các file thời tiết hàng ngày (date theo ngày)

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

# Paths
PROJECT_ROOT = Path.cwd().parent.parent
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
SALINITY_FILE = PROCESSED_DIR / 'landsat_salinity_2022_full_year' / 'salinity_h3_features_2022.csv'

print(f"Project Root: {PROJECT_ROOT}")
print(f"Salinity file exists: {SALINITY_FILE.exists()}")

Project Root: d:\Mekong_DGGS
Salinity file exists: True


In [2]:
# Load salinity data (monthly)
df_salinity = pd.read_csv(SALINITY_FILE)
print(f"Salinity shape: {df_salinity.shape}")
print(f"Salinity columns: {df_salinity.columns.tolist()}")
print(f"\nTime values (months): {sorted(df_salinity['time'].unique())}")
print(f"\nSalinity sample:")
df_salinity.head()

Salinity shape: (83508, 10)
Salinity columns: ['h3_index', 'time', 'salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std', 'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']

Time values (months): [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]

Salinity sample:


,h3_index,time,salinity_min,salinity_max,salinity_mean,salinity_std,pct_salinity_pixels,pct_salinity_gte_0_2,pct_salinity_gte_0_5,pct_salinity_gte_1_0
0,8765b5371ffffff,1,0.122681,0.789567,0.597343,0.104642,55.434343,99.872449,82.197522,0.0
1,8765b5262ffffff,1,-0.052238,0.725607,0.394314,0.098543,67.690443,97.656716,13.671642,0.0
2,8765a2374ffffff,1,0.075732,0.791306,0.681155,0.087410,68.434343,99.955720,94.583026,0.0
3,8765a35b4ffffff,1,0.186754,0.745908,0.616323,0.049406,68.646465,99.970571,97.027663,0.0
4,8765a628effffff,1,0.006427,0.837769,0.527763,0.149161,67.848485,97.841298,61.813310,0.0


In [3]:
# Load daily weather data (example: temperature)
DAILY_FILE = PROCESSED_DIR / 'h3_temp_daily_filled.csv'

# Read with chunking if file is large
print(f"Loading {DAILY_FILE.name}...")
df_daily = pd.read_csv(DAILY_FILE)
print(f"Daily data shape: {df_daily.shape}")
print(f"Daily columns: {df_daily.columns.tolist()}")
print(f"\nDaily sample:")
df_daily.head()

Loading h3_temp_daily_filled.csv...
Daily data shape: (2540035, 3)
Daily columns: ['h3_index', 'date', 'temp_c']

Daily sample:


,h3_index,date,temp_c
0,8765a035dffffff,2025-01-01,26.275557
1,8765a3634ffffff,2025-01-01,26.304203
2,8765b5229ffffff,2025-01-01,25.848473
3,876584d4bffffff,2025-01-01,26.401371
4,8765a66e1ffffff,2025-01-01,25.796797


In [4]:
# Convert daily date to month for matching
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_daily['month'] = df_daily['date'].dt.month
df_daily['year'] = df_daily['date'].dt.year

print(f"Date range: {df_daily['date'].min()} to {df_daily['date'].max()}")
print(f"\nSample with month:")
df_daily[['h3_index', 'date', 'month', 'year']].head(10)

Date range: 2025-01-01 00:00:00 to 2025-12-31 00:00:00

Sample with month:


,h3_index,date,month,year
0,8765a035dffffff,2025-01-01,1,2025
1,8765a3634ffffff,2025-01-01,1,2025
2,8765b5229ffffff,2025-01-01,1,2025
3,876584d4bffffff,2025-01-01,1,2025
4,8765a66e1ffffff,2025-01-01,1,2025
5,8765a2046ffffff,2025-01-01,1,2025
6,8765a2086ffffff,2025-01-01,1,2025
7,8765a2b1affffff,2025-01-01,1,2025
8,8765a04a2ffffff,2025-01-01,1,2025
9,8765a66cbffffff,2025-01-01,1,2025


In [5]:
# Filter daily data to 2022 only (to match salinity year)
df_daily_2022 = df_daily[df_daily['year'] == 2022].copy()
print(f"Daily 2022 shape: {df_daily_2022.shape}")
print(f"Unique months in 2022: {sorted(df_daily_2022['month'].unique())}")

Daily 2022 shape: (0, 5)
Unique months in 2022: []


In [6]:
# Merge salinity (monthly) with daily data
# Strategy: Join on h3_index + month
df_merged = df_daily_2022.merge(
    df_salinity,
    left_on=['h3_index', 'month'],
    right_on=['h3_index', 'time'],
    how='inner'
)

print(f"Merged shape: {df_merged.shape}")
print(f"Merged columns: {df_merged.columns.tolist()}")
print(f"\nMerged sample:")
df_merged.head()

Merged shape: (0, 14)
Merged columns: ['h3_index', 'date', 'temp_c', 'month', 'year', 'time', 'salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std', 'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']

Merged sample:


,h3_index,date,temp_c,month,year,time,salinity_min,salinity_max,salinity_mean,salinity_std,pct_salinity_pixels,pct_salinity_gte_0_2,pct_salinity_gte_0_5,pct_salinity_gte_1_0


In [7]:
# Clean up columns
# Drop redundant 'time' column (duplicate of 'month')
df_merged = df_merged.drop(columns=['time', 'year', 'month'])

# Reorder columns: h3_index, date, weather features, salinity features
cols_order = ['h3_index', 'date']
weather_cols = [c for c in df_merged.columns if c.startswith(('temp', 'rain', 'rh', 'solar')) or c in ['temp_c']]
salinity_cols = [c for c in df_merged.columns if c.startswith('salinity') or c.startswith('pct_salinity')]
other_cols = [c for c in df_merged.columns if c not in cols_order + weather_cols + salinity_cols]

df_merged = df_merged[cols_order + weather_cols + salinity_cols + other_cols]

print(f"Final columns: {df_merged.columns.tolist()}")
print(f"\nFinal data preview:")
df_merged.head(10)

Final columns: ['h3_index', 'date', 'temp_c', 'salinity_min', 'salinity_max', 'salinity_mean', 'salinity_std', 'pct_salinity_pixels', 'pct_salinity_gte_0_2', 'pct_salinity_gte_0_5', 'pct_salinity_gte_1_0']

Final data preview:


,h3_index,date,temp_c,salinity_min,salinity_max,salinity_mean,salinity_std,pct_salinity_pixels,pct_salinity_gte_0_2,pct_salinity_gte_0_5,pct_salinity_gte_1_0


In [8]:
# Check data quality
print("Data Quality Check:")
print(f"Total rows: {len(df_merged):,}")
print(f"Unique H3 cells: {df_merged['h3_index'].nunique():,}")
print(f"Date range: {df_merged['date'].min()} to {df_merged['date'].max()}")
print(f"\nMissing values:")
print(df_merged.isnull().sum()[df_merged.isnull().sum() > 0])
print(f"\nBasic statistics:")
df_merged.describe()

Data Quality Check:
Total rows: 0
Unique H3 cells: 0
Date range: NaT to NaT

Missing values:
Series([], dtype: int64)

Basic statistics:


,date,temp_c,salinity_min,salinity_max,salinity_mean,salinity_std,pct_salinity_pixels,pct_salinity_gte_0_2,pct_salinity_gte_0_5,pct_salinity_gte_1_0
count,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Save to CSV
OUTPUT_FILE = PROCESSED_DIR / 'merged_temp_salinity_2022.csv'
df_merged.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Saved to: {OUTPUT_FILE}")
print(f"File size: {OUTPUT_FILE.stat().st_size / 1024 / 1024:.2f} MB")

✅ Saved to: d:\Mekong_DGGS\data\processed\merged_temp_salinity_2022.csv
File size: 0.00 MB
